# Walmart sales forecasting using RNN (LSTM )
1. data preprocessing.
 
 Some things to consider when cleaning. we have to map null values to0. We also have to make sure to parse date time correctly. 

2. visualize the data to get a general idea


# References:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import os

# 1. Data Preprocessing 

In [ ]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
features = pd.read_csv('data/features.csv')
stores = pd.read_csv('data/stores.csv')

train = train.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
train = train.merge(stores, on="Store", how="left")

test = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
test = test.merge(stores, on="Store", how="left")

for df in [train, test]:
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week'] = df['Date'].dt.isocalendar().week.astype(int)
    df['Day'] = df['Date'].dt.dayofweek

train['Type'] = train['Type'].map({'A': 0, 'B': 1, 'C': 2})
test['Type'] = test['Type'].map({'A': 0, 'B': 1, 'C': 2})


features_cols = ['Store', 'Dept', 'IsHoliday', 'Temperature', 'Fuel_Price',
                 'CPI', 'Unemployment', 'Size', 'Type',
                 'Year', 'Month', 'Week', 'Day']

X_train = train[features_cols]
y_train = train['Weekly_Sales']



In [ ]:
batch_size=32
seq_len = 7 # for 7 days a week
input_size = 13 # since we have 13 numerical features after data cleaning

# LSTM Implementation

In [ ]:
class ForecastLSTM(nn.Module):
   
    def __init__(self, input_dim, hidden_dim, num_layers,output_dim, bias=True):
        super(ForecastLSTM, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_dim,hidden_dim,num_layers, bias)
        self.fc = nn.Linear(hidden_dim,output_dim)

    def forward(self, x):
        device = x.device
        ht = torch.zeros(self.hidden_dim,device=device)
        ct = torch.zeros(self.hidden_dim,device=device)

        output, _ = self.lstm_cell(x[i], ht, ct)

        output = self.fc(output[:,-1,:])
        return output
    

## Training and Testing Implementaiton

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

In [ ]:
#hyperparameters: 
num_epochs = 100

input_dim = 13
hidden_dim = 64
num_layers = 1
output_dim = 1
learning_rate = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ForecastLSTM(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


hist = np.zeros(num_epochs)
start_time = time.time()

for t in range(num_epochs):
    x_train = x_train.to(device)
    y_train = y_train.to(device)
    y_train_pred = model(x_train)

    loss = criterion(y_train_pred, y_train.view(1,-1))
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

training_time = time.time()-start_time
print("Training time: {}".format(training_time))